* https://huggingface.co/learn/nlp-course/en/chapter5/5?fw=pt
* https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter5/section5.ipynb

In [3]:
!pip install datasets evaluate transformers[sentencepiece]
!sudo apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 3544 kB of archives.
After this operation, 10.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 git-lfs amd64 3.0.2-1ubuntu0.3 [3544 kB]
Fetched 3544 kB in 1s (2609 kB/s)  

78Selecting previously unselected package git-lfs.
(Reading database ... 66234 files and directories currently installed.)
Preparing to unpack .../git-lfs_3.0.2-1ubuntu0.3_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking git-lfs (3.0.2-1ubuntu0.3) ...
7Progress: [ 40%] [#######################...................................] 8Setting up git-lfs (3.0.2-1ubuntu0.3) ...
7Progress: [ 80%] 

In [4]:
!git config --global user.email "renwei2004@gmail.com"
!git config --global user.name "Wei Ren"

In [5]:
from huggingface_hub import notebook_login

notebook_login()

# Get the Github issues

In [6]:
!pip install requests

In [10]:
import requests

url = "https://api.github.com/repos/NVIDIA/NeMo/issues?page=1&per_page=1"
response = requests.get(url)

In [11]:
response.status_code

200

In [12]:
response.json()

[{'url': 'https://api.github.com/repos/NVIDIA/NeMo/issues/11904',
  'repository_url': 'https://api.github.com/repos/NVIDIA/NeMo',
  'labels_url': 'https://api.github.com/repos/NVIDIA/NeMo/issues/11904/labels{/name}',
  'comments_url': 'https://api.github.com/repos/NVIDIA/NeMo/issues/11904/comments',
  'events_url': 'https://api.github.com/repos/NVIDIA/NeMo/issues/11904/events',
  'html_url': 'https://github.com/NVIDIA/NeMo/pull/11904',
  'id': 2800382613,
  'node_id': 'PR_kwDOC_bI7s6IanOx',
  'number': 11904,
  'title': 'remove nemo.collection.nlp imports from nemo2',
  'user': {'login': 'dimapihtar',
   'id': 37850217,
   'node_id': 'MDQ6VXNlcjM3ODUwMjE3',
   'avatar_url': 'https://avatars.githubusercontent.com/u/37850217?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/dimapihtar',
   'html_url': 'https://github.com/dimapihtar',
   'followers_url': 'https://api.github.com/users/dimapihtar/followers',
   'following_url': 'https://api.github.com/users/dimapihtar/foll

In [13]:
!pip install python-dotenv

In [19]:
# https://github.com/theskumar/python-dotenvabs
%load_ext dotenv
%dotenv ../.env

In [20]:
import os

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [17]:
import math
import time
from pathlib import Path

import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [18]:
fetch_issues(owner="NVIDIA", repo="NeMo")

  0%|          | 0/100 [00:00<?, ?it/s]

Reached GitHub rate limit. Sleeping for one hour ...
Downloaded all the issues for NeMo! Dataset stored at ./NeMo-issues.jsonl


# Clean up the data

In [71]:
# We hit a bug about inferred schema: https://github.com/huggingface/datasets/issues/7222.
# We create a new jsonl file which consists of the last 1000 lines and then the first 9000 lines of NeMo-issues.jsonl.
!(tail -n 1000 NeMo-issues.jsonl && head -n 9000 NeMo-issues.jsonl) > NeMo-issues-fixed.jsonl

In [74]:
from datasets import load_dataset

issues_dataset = load_dataset(
    "json", data_files="NeMo-issues-fixed.jsonl", split="train"
)
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason'],
    num_rows: 10000
})

In [76]:
sample = issues_dataset.shuffle(seed=24).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/NVIDIA/NeMo/pull/6899
>> Pull request: {'url': 'https://api.github.com/repos/NVIDIA/NeMo/pulls/6899', 'html_url': 'https://github.com/NVIDIA/NeMo/pull/6899', 'diff_url': 'https://github.com/NVIDIA/NeMo/pull/6899.diff', 'patch_url': 'https://github.com/NVIDIA/NeMo/pull/6899.patch', 'merged_at': datetime.datetime(2023, 6, 21, 21, 41, 32)}

>> URL: https://github.com/NVIDIA/NeMo/pull/7535
>> Pull request: {'url': 'https://api.github.com/repos/NVIDIA/NeMo/pulls/7535', 'html_url': 'https://github.com/NVIDIA/NeMo/pull/7535', 'diff_url': 'https://github.com/NVIDIA/NeMo/pull/7535.diff', 'patch_url': 'https://github.com/NVIDIA/NeMo/pull/7535.patch', 'merged_at': datetime.datetime(2023, 9, 27, 11, 56, 53)}

>> URL: https://github.com/NVIDIA/NeMo/issues/2524
>> Pull request: None



In [77]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Augment the dataset

In [78]:
issue_number = 2524
url = f"https://api.github.com/repos/NVIDIA/NeMo/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/NVIDIA/NeMo/issues/comments/884570079',
  'html_url': 'https://github.com/NVIDIA/NeMo/issues/2524#issuecomment-884570079',
  'issue_url': 'https://api.github.com/repos/NVIDIA/NeMo/issues/2524',
  'id': 884570079,
  'node_id': 'IC_kwDOC_bI7s40uXff',
  'user': {'login': 'VahidooX',
   'id': 23551647,
   'node_id': 'MDQ6VXNlcjIzNTUxNjQ3',
   'avatar_url': 'https://avatars.githubusercontent.com/u/23551647?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/VahidooX',
   'html_url': 'https://github.com/VahidooX',
   'followers_url': 'https://api.github.com/users/VahidooX/followers',
   'following_url': 'https://api.github.com/users/VahidooX/following{/other_user}',
   'gists_url': 'https://api.github.com/users/VahidooX/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/VahidooX/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/VahidooX/subscriptions',
   'organizations_url': 'https://api.githu

In [81]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/NVIDIA/NeMo/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    if response.status_code == requests.codes.ok:
        return [r["body"] for r in response.json()]
    else:
        return None


# Test our function works as expected
get_comments(2524)

['It looks like you are an older version of python, probably even python 2. You need to install the latest python or conda.',
 "closing this. Please follow one of these installation instructions https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/v1.1.0/starthere/intro.html#installation  Feel free to re-open if they don't work.\r\n"]

In [82]:
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Upload the dataset to Hugging Face

In [86]:
from huggingface_hub import notebook_login

notebook_login()

In [84]:
issues_with_comments_dataset.push_to_hub("nemo-github-issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/renwei2024/nemo-github-issues/commit/862dc8c8f1688b8717e393d5293d54b79c4467f2', commit_message='Upload dataset', commit_description='', oid='862dc8c8f1688b8717e393d5293d54b79c4467f2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/renwei2024/nemo-github-issues', endpoint='https://huggingface.co', repo_type='dataset', repo_id='renwei2024/nemo-github-issues'), pr_revision=None, pr_num=None)

In [87]:
remote_dataset = load_dataset("renwei2024/nemo-github-issues", split="train")
remote_dataset

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'sub_issues_summary', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'is_pull_request'],
    num_rows: 10000
})